# GWAS 

Original data was pulled from [here](https://www.ebi.ac.uk/gwas/docs/file-downloads) (all associations v1.0) on 2/14/24.

In [1]:
import pandas as pd
import table_cleaning_functions as tcf 

# read in raw data 
gwas_raw = pd.read_csv('../home/raw_data/GlyGen/gwas_raw.tsv', sep='\t')

/tmp/ipykernel_1743/2536380011.py:5: DtypeWarning: Columns (9,11,12,13,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  gwas_raw = pd.read_csv('../home/raw_data/GlyGen/gwas_raw.tsv', sep='\t')


In [2]:
print(gwas_raw.shape)
gwas_raw.head()

(571148, 34)


,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,CONTEXT,INTERGENIC,RISK ALLELE FREQUENCY,P-VALUE,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV
0,2017-08-10,28443625,Justice AE,2017-04-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28443625,"Genome-wide meta-analysis of 241,258 adults ac...",BMI in non-smokers,"77,279 European ancestry women, 47,280 Europea...","16,011 European ancestry women, 17,912 Europea...",...,intergenic_variant,1.0,0.847,5.000000e-08,7.301030,NaN,0.0230,[0.015-0.031] kg/m2 increase,"Affymetrix, Illumina, Perlegen [up to 2800000]...",N
1,2017-08-10,28443625,Justice AE,2017-04-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28443625,"Genome-wide meta-analysis of 241,258 adults ac...",BMI in non-smokers,"77,279 European ancestry women, 47,280 Europea...","16,011 European ancestry women, 17,912 Europea...",...,intergenic_variant,1.0,0.469,7.000000e-07,6.154902,NaN,0.0192,[0.012-0.027] kg/m2 increase,"Affymetrix, Illumina, Perlegen [up to 2800000]...",N
2,2017-08-10,28443625,Justice AE,2017-04-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28443625,"Genome-wide meta-analysis of 241,258 adults ac...",BMI in non-smokers,"77,279 European ancestry women, 47,280 Europea...","16,011 European ancestry women, 17,912 Europea...",...,intron_variant,0.0,0.2894,7.000000e-09,8.154902,(men),0.0341,[0.023-0.046] kg/m2 decrease,"Affymetrix, Illumina, Perlegen [up to 2800000]...",N
3,2017-08-10,28443625,Justice AE,2017-04-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28443625,"Genome-wide meta-analysis of 241,258 adults ac...",BMI in non-smokers,"77,279 European ancestry women, 47,280 Europea...","16,011 European ancestry women, 17,912 Europea...",...,intron_variant,0.0,0.2894,7.000000e-10,9.154902,NaN,0.0264,[0.018-0.035] kg/m2 decrease,"Affymetrix, Illumina, Perlegen [up to 2800000]...",N
4,2017-08-10,28443625,Justice AE,2017-04-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28443625,"Genome-wide meta-analysis of 241,258 adults ac...",BMI in non-smokers,"77,279 European ancestry women, 47,280 Europea...","16,011 European ancestry women, 17,912 Europea...",...,intergenic_variant,1.0,0.8269,3.000000e-09,8.522879,(women),0.0376,[0.025-0.05] kg/m2 decrease,"Affymetrix, Illumina, Perlegen [up to 2800000]...",N


Drop unnecessary columns. 

In [3]:
gwas_raw.columns

Index(['DATE ADDED TO CATALOG', 'PUBMEDID', 'FIRST AUTHOR', 'DATE', 'JOURNAL',
       'LINK', 'STUDY', 'DISEASE/TRAIT', 'INITIAL SAMPLE SIZE',
       'REPLICATION SAMPLE SIZE', 'REGION', 'CHR_ID', 'CHR_POS',
       'REPORTED GENE(S)', 'MAPPED_GENE', 'UPSTREAM_GENE_ID',
       'DOWNSTREAM_GENE_ID', 'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE',
       'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE', 'SNPS',
       'MERGED', 'SNP_ID_CURRENT', 'CONTEXT', 'INTERGENIC',
       'RISK ALLELE FREQUENCY', 'P-VALUE', 'PVALUE_MLOG', 'P-VALUE (TEXT)',
       'OR or BETA', '95% CI (TEXT)', 'PLATFORM [SNPS PASSING QC]', 'CNV'],
      dtype='object')

In [4]:
drop_cols = ['DATE ADDED TO CATALOG', 'PUBMEDID', 'FIRST AUTHOR', 'DATE', 'JOURNAL',
            'LINK', 'STUDY', 'INITIAL SAMPLE SIZE', 'REPLICATION SAMPLE SIZE', 'REGION',
            'CHR_ID', 'CHR_POS', 'REPORTED GENE(S)', 'UPSTREAM_GENE_ID', 'DOWNSTREAM_GENE_ID',
            'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE', 'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE',
            'MERGED', 'SNP_ID_CURRENT', 'CONTEXT', 'INTERGENIC', 'RISK ALLELE FREQUENCY', 'P-VALUE',
            'PVALUE_MLOG', 'P-VALUE (TEXT)', 'OR or BETA', '95% CI (TEXT)', 'PLATFORM [SNPS PASSING QC]', 'CNV']
gwas_raw = gwas_raw.drop(columns = drop_cols)
gwas_raw.head()

,DISEASE/TRAIT,MAPPED_GENE,SNPS
0,BMI in non-smokers,SRRM1P2 - LINC00971,rs6794880
1,BMI in non-smokers,STARP1 - HNRNPA3P5,rs9540493
2,BMI in non-smokers,TCF7L2,rs7903146
3,BMI in non-smokers,TCF7L2,rs7903146
4,BMI in non-smokers,RPS17P5 - FTH1P5,rs2207139


Only interested in cancer related rows right now, filter on cancer related conditions. 

In [8]:
# cancer related words
cancer_related = ['cancer', 'carcinoma', 'leukemia', 'tumor', 'malignancy', 'glioblastoma',
                'melanoma', 'lymphoma', 'sarcoma']

# filter on only cancer related rows
gwas_raw = gwas_raw[gwas_raw['DISEASE/TRAIT'].str.contains('|'.join(cancer_related), case = False, na = False)]
print(gwas_raw.shape)
gwas_raw.head()

(15456, 3)


,DISEASE/TRAIT,MAPPED_GENE,SNPS
149,Survival in pancreatic cancer,PAIP2B,rs113988120
150,Survival in pancreatic cancer,"B4GALT4, B4GALT4-AS1",rs4568126
151,Survival in pancreatic cancer,RPL21P119 - LINC02177,rs4780973
152,Survival in pancreatic cancer,LINC00376 - LINC00395,rs1000589
153,Survival in pancreatic cancer,LINC01163 - LINC02667,rs10734079


Drop range, list, and null mapped gene rows. 

In [12]:
# drop rows where MAPPED_GENE is NA
gwas_raw = gwas_raw.dropna(subset = ['MAPPED_GENE'])
gwas_raw['MAPPED_GENE'].isna().sum()

0

In [15]:
# drop rows where MAPPED_GENE is a range
gwas_raw = gwas_raw[~gwas_raw['MAPPED_GENE'].str.contains(' - ')]
gwas_raw.shape

(9394, 3)

In [18]:
# drop rows where MAPPED_GENE is a list
gwas_raw = gwas_raw[~gwas_raw['MAPPED_GENE'].str.contains(',')]
gwas_raw.shape

(8512, 3)

In [19]:
gwas_raw.head()

,DISEASE/TRAIT,MAPPED_GENE,SNPS
149,Survival in pancreatic cancer,PAIP2B,rs113988120
154,Survival in pancreatic cancer,ACOT11,rs10736390
157,Survival in pancreatic cancer,BDNF-AS,rs10767646
158,Survival in pancreatic cancer,WHRN,rs10817611
161,Survival in pancreatic cancer,LIN7C,rs10835188


In [22]:
# save to tsv 
gwas_raw.to_csv('../home/data/processed_data/gwas_processed.tsv', sep = '\t', index = False)

In [24]:
# isolate conditions
conditions = gwas_raw['DISEASE/TRAIT']
# consolidate unique conditions
conditions = conditions.drop_duplicates()
conditions.to_csv('../home/data/processed_data/gwas_conditions.tsv', sep = '\t', index = False)

Create the empty TSV file.

In [6]:
df = pd.DataFrame(columns = tcf.TSV_HEADERS)
df.head()

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag
